In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from matplotlib import pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("/kaggle/input/cross-site-scripting-xss-dataset-for-deep-learning/XSS_dataset.csv")

In [3]:
X = csr_matrix(df.drop(['Sentence'], axis=1).values)
y = df['Label'].values

In [4]:
X = X.toarray()